<a href="https://colab.research.google.com/github/Shivyogi26/HTML.Portfolio/blob/main/DataAngles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests
!pip install pandas
!pip install numpy
!pip install pillow
!pip install opencv-python scikit-learn pandas matplotlib
!pip install pandas numpy requests tensorflow pillow scikit-learn

In [3]:
import pandas as pd

# Load train and test datasets
train_df = pd.read_csv('/content/sample_data/data set/test.csv')
test_df = pd.read_csv('/content/sample_data/data set/train.csv')

# Inspect the data
print(train_df.head())
print(test_df.head())

   index                                         image_link  group_id  \
0      0  https://m.media-amazon.com/images/I/110EibNycl...    156839   
1      1  https://m.media-amazon.com/images/I/11TU2clswz...    792578   
2      2  https://m.media-amazon.com/images/I/11TU2clswz...    792578   
3      3  https://m.media-amazon.com/images/I/11TU2clswz...    792578   
4      4  https://m.media-amazon.com/images/I/11gHj8dhhr...    792578   

  entity_name  
0      height  
1       width  
2      height  
3       depth  
4       depth  
                                          image_link  group_id  entity_name  \
0  https://m.media-amazon.com/images/I/61I9XdN6OF...    748919  item_weight   
1  https://m.media-amazon.com/images/I/71gSRbyXmo...    916768  item_volume   
2  https://m.media-amazon.com/images/I/61BZ4zrjZX...    459516  item_weight   
3  https://m.media-amazon.com/images/I/612mrlqiI4...    459516  item_weight   
4  https://m.media-amazon.com/images/I/617Tl40LOX...    731432  item_w

In [5]:
import requests
from PIL import Image
from io import BytesIO
import os

# Create a directory to save images
if not os.path.exists('images'):
    os.makedirs('images')

def download_image(image_url, image_name):
    try:
        response = requests.get(image_url)
        img = Image.open(BytesIO(response.content))
        img.save(f'images/{image_name}.jpg')
        print(f"Downloaded: {image_name}")
    except Exception as e:
        print(f"Failed to download {image_url}: {e}")

# Example to download all images
for idx, row in train_df.iterrows():
    download_image(row['image_link'], row['index'])

Streaming output truncated to the last 5000 lines.
Downloaded: 126283
Downloaded: 126284
Downloaded: 126285
Downloaded: 126286
Downloaded: 126287
Downloaded: 126288
Downloaded: 126289
Downloaded: 126290
Downloaded: 126291
Downloaded: 126292
Downloaded: 126293
Downloaded: 126294
Downloaded: 126295
Downloaded: 126296
Downloaded: 126297
Downloaded: 126298
Downloaded: 126299
Downloaded: 126300
Downloaded: 126301
Downloaded: 126302
Downloaded: 126303
Downloaded: 126304
Downloaded: 126305
Downloaded: 126306
Downloaded: 126307
Downloaded: 126308
Downloaded: 126309
Downloaded: 126310
Downloaded: 126311
Downloaded: 126312
Downloaded: 126313
Downloaded: 126314
Downloaded: 126315
Downloaded: 126316
Downloaded: 126317
Downloaded: 126318
Downloaded: 126319
Downloaded: 126320
Downloaded: 126321
Downloaded: 126322
Downloaded: 126323
Downloaded: 126324
Downloaded: 126325
Downloaded: 126326
Downloaded: 126327
Downloaded: 126328
Downloaded: 126329
Downloaded: 126330
Downloaded: 126331
Downloaded: 126332

In [6]:
import tensorflow as tf

def preprocess_image(image_path, target_size=(224, 224)):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=target_size)
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize pixel values
    return img_array

# Example to preprocess an image
image_array = preprocess_image('images/1.jpg')

In [7]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model

# Load a pre-trained model (ResNet50 in this case)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model
base_model.trainable = False

# Add custom layers on top
x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
output = Dense(1, activation='linear')(x)  # For predicting numeric entity value

# Build the model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [14]:
import numpy as np

# Prepare the input images and labels
X_train = []

# Access the 'entity_value' column from the correct DataFrame (test_df)
y_train = test_df['entity_value'].apply(lambda x: float(x.split()[0]))  # Extract numeric value

for idx in test_df['index']: # Iterate over the index from the correct DataFrame
    image_path = f'images/{idx}.jpg'
    X_train.append(preprocess_image(image_path))

X_train = np.vstack(X_train)
y_train = y_train.values

ValueError: could not convert string to float: '[100.0,'

In [15]:
print(test_df.columns)  # Check if 'entity_value' exists
print(test_df['entity_value'].head())  # Check the values in 'entity_value'


Index(['image_link', 'group_id', 'entity_name', 'entity_value'], dtype='object')
0        500.0 gram
1           1.0 cup
2        0.709 gram
3        0.709 gram
4    1400 milligram
Name: entity_value, dtype: object


In [17]:
import numpy as np

# Function to clean and extract the numeric part
def clean_entity_value(value):
    try:
        # Split the string by spaces, take the first part, remove commas and brackets
        clean_value = value.split()[0].replace(',', '').replace('[', '').replace(']', '')
        return float(clean_value)  # Convert to float
    except ValueError as e:
        print(f"Could not convert {value} to float: {e}")
        return np.nan  # Handle cases where conversion fails

# Clean and convert 'entity_value' column
y_train = test_df['entity_value'].apply(clean_entity_value)

# Drop any rows where conversion failed (optional)
y_train = y_train.dropna()

# Ensure 'index' exists and iterate over it
for idx in test_df['index']:  # Or use test_df.index if 'index' is not a column
    image_path = f'images/{idx}.jpg'
    try:
        X_train.append(preprocess_image(image_path))
    except Exception as e:
        print(f"Error processing {image_path}: {e}")

# Stack images after ensuring all have the same shape
X_train = np.vstack(X_train)

# Convert y_train to a NumPy array
y_train = y_train.values


KeyError: 'index'

In [16]:
import numpy as np

# Prepare the input images and labels
X_train = []

# Check if 'entity_value' exists and can be processed
print(test_df.columns)  # Debug: Check if 'entity_value' exists
print(test_df['entity_value'].head())  # Debug: Check the first few values of 'entity_value'

# Access the 'entity_value' column and extract numeric values
y_train = test_df['entity_value'].apply(lambda x: float(x.split()[0]))

# Ensure 'index' exists and is valid
print(test_df['index'].head())  # Debug: Check the 'index' column values

# Iterate over the 'index' column and load images
for idx in test_df['index']:  # Or use test_df.index if 'index' is not a column
    image_path = f'images/{idx}.jpg'
    try:
        X_train.append(preprocess_image(image_path))
    except Exception as e:
        print(f"Error processing {image_path}: {e}")

# Ensure all images have the same shape before stacking
print([img.shape for img in X_train])  # Debug: Check if all images have the same shape

# Stack the images into a single array
X_train = np.vstack(X_train)

# Convert y_train to a NumPy array
y_train = y_train.values


Index(['image_link', 'group_id', 'entity_name', 'entity_value'], dtype='object')
0        500.0 gram
1           1.0 cup
2        0.709 gram
3        0.709 gram
4    1400 milligram
Name: entity_value, dtype: object


ValueError: could not convert string to float: '[100.0,'

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)